In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression

CNT_MODELS = 30 # 30 linear models

In [ ]:
%%time
data = pd.read_pickle('../input/ump-train-picklefile/train.pkl')

y = data.target.copy()
X = data.drop(columns=['time_id', 'target'])
X.row_id = pd.to_numeric(X.row_id.str.split('_').map(lambda x: x[1]))

Обучим `CNT_MODELS` линейных моделей (у меня 30 моделей).

И при предсказании найдем среднее предскзаание всех моделей.

In [ ]:
models = []

for i in range(0, CNT_MODELS):
    X_s = X.loc[i::CNT_MODELS]
    y_s = y.loc[X_s.index]
    models.append(LinearRegression().fit(X_s, y_s))

In [ ]:
class LINEAR_models():
    
    def __init__(self, models):
        self.models = models
        
    def predict(self, X):
        summ = self.models[0].predict(X)
        for model in self.models[1:]:
            summ += model.predict(X)
            
        return summ / len(self.models)
    
model = LINEAR_models(models)

In [ ]:
import ubiquant
env = ubiquant.make_env()
iter_test = env.iter_test()

for (test_df, sample_prediction_df) in iter_test:
    test_df.row_id = pd.to_numeric(test_df.row_id.str.split('_').map(lambda x: x[1]))
    sample_prediction_df['target'] = model.predict(test_df)
    env.predict(sample_prediction_df)
    